In [1]:
import requests
from datetime import datetime
import pandas as pd
from pytz import timezone
import time
import warnings
warnings.filterwarnings('ignore')

# 0. 데이터 불러오기

In [2]:
from google.colab import drive
drive.mount('/gdrive')

path = ""
df_prod = pd.read_excel(f"{path}크롤링상품리스트.xlsx", usecols=[2,4,5,6,7,8,9], header=1)
# df_prod

Mounted at /gdrive


,스타일코드,아이템대분류,아이템중분류,정상가,입고량,재고량,원가율
0,SPRPD37C02,티셔츠류,런닝형티셔츠,19900,66300,40168,0.279243
1,SPRLD37U05,패션잡화,지갑,19900,32469,7338,0.286759
2,SPYWD25C01,셔츠류,남방,35900,30284,8966,0.251786
3,SPKWD25U03,스웨터류,스웨터,29900,50292,21676,0.348862
4,SPTJD25C51,바지류,청바지,49900,30277,10880,0.246208
...,...,...,...,...,...,...,...
195,SPTAD24M02,바지류,바지,49900,7671,4957,0.281704
196,SPKWD25W14,스웨터류,스웨터,39900,8364,5352,0.250173
197,SPCKD25W24,스웨터류,스웨터가디건,39900,4749,2412,0.260807
198,SPRWD23U02,티셔츠류,런닝형티셔츠,19900,30910,19315,0.242231


# 1. 전체상품(df_total)

In [ ]:
headers = {'Referer':'https://search.shopping.naver.com/catalog/'
          ,'Sbth':'0b003b74b1ac4ff7fd4f6aeb333912dd8d0b78ec95e97e0b43eb7fdf079f6283c3d6cac389fcbc07608dc240e3a5ea90'
          ,'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
           }

date = datetime.now(timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M")

In [ ]:
# 전체상품 불러오는 함수
# pagingIndex=80, sort=rel
# 해외직구제외(exagency), 중고제외(exused), 렌탈제외(exrental)

def productSettotal(style_code):
    df = []
    pages = 1

    while True:
        url = f"https://search.shopping.naver.com/api/search/all?eq=&exagency=true&exrental=true&exused=true&iq=&origQuery={style_code}&pagingIndex={pages}&pagingSize=80&productSet=total&query={style_code}&sort=rel&sps=Y&viewType=list&xq="
        response = requests.get(url,headers=headers)
        temp = response.json()['shoppingResult']

        if temp['productCount'] == 0:
            break;

        df = df + temp["products"]
        pages = pages + 1

    return pd.DataFrame(df)

In [ ]:
df_total = pd.DataFrame()

for index, row in df_prod.iterrows():
    styleCode = row.스타일코드

    df_temp = productSettotal(styleCode)
    # print(styleCode)
    time.sleep(3)
    df_temp["styleCode"] = styleCode
    df_temp["timestamp"] = datetime.now(timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M")

    df_total = pd.concat([df_total, df_temp])

SPRPD37C02
SPRLD37U05
SPYWD25C01
SPKWD25U03
SPTJD25C51
SPPPD37KU4
SPTJD25G51
SPJJD25C21
SPPPD37U04
SPKWD24M01
SPMTD26U01
SPPPD25U04
SPJJD38G01
SPPPD37U03
SPJJD25G01
SPTCD25C11
SPYWD25G01
SPRWDA9U01
SPPPD25U01
SPPPD24KU2
SPPPD25KU4
SPPPD37U06
SPJJD37C21
SPPPD25U03
SPPPD25U07
SPTCD24C01
SPYWD25C02
SPYJD26C01
SPTJD25C21
SPTJD25G31
SPTJD37G51
SPRPD37C03
SPTJDA9G51
SPTCD25G01
SPTND26G11
SPTJD37C51
SPYWD23C01
SPRPD25G12
SPHWD24U01
SPTAD25M06
SPJJD23G01
SPPPD25U10
SPPPD25U06
SPHWD24G01
SPRWD24M01
SPJJD23C21
SPRWDA9U03
SPRPD26C11
SPKWD25U12
SPTHD25C22
SPYAD26W01
SPRPD24G11
SPRPD24C02
SPKWD25U11
SPMTD24U04
SPRPD24C04
SPRPD24G12
SPYAD24W11
SPHWD24C21
SPRPD37G10
SPPPD25U09
SPTHD26C11
SPTAD24M01
SPMTD37U06
SPYWD24M01
SPRPD25G13
SPPPD25U02
SPKWD25U02
SPRPD37C22
SPRPD37G13
SPWHD24KG1
SPOMD24U03
SPCKD25U02
SPTJD37C52
SPTHD26C01
SPRPD25M04
SPTHD25G01
SPCKD37G01
SPWHD37W23
SPJKD25W11
SPJKD24W11
SPTHD37G01
SPLWD38G01
SPRPD37C91
SPYWD25G02
SPWJD38G02
SPMTD37U02
SPTAD24W01
SPJJD25M01
SPMTD25G01
SPTJD26C55

In [ ]:
df_lowMallList = df_total.loc[(df_total["lowMallList"].isnull() == False)]
# df_lowMallList

,collection,purchaseConditionInfos,rank,id,parentId,hasLowestCardPrice,hasPromotionCont,hasAddInFee,scoreInfo,category1Id,...,lowPriceByMallNo,shopNNo,dummy,lowMallList,mallInfoCache,crUrl,crUrlMore,styleCode,timestamp,channelInfoCache
0,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",1.0,40127022611,40127022611,1.0,0.0,0.0,NaN,50000000,...,40457021662/17703/5878053145/17270//3000//|404...,0,,"[{'nvMid': '40457021662', 'mallSeq': '17703', ...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=v8wUm...,https://cr.shopping.naver.com/adcr.nhn?x=mQvGx...,SPRLD37U05,2023-06-27 19:49,NaN
1,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",2.0,40772477650,40772477650,1.0,0.0,0.0,NaN,50000000,...,40324513511/221844/20598002566/17340//3000//|4...,0,,"[{'nvMid': '40324513511', 'mallSeq': '221844',...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=v7%2B...,https://cr.shopping.naver.com/adcr.nhn?x=Ze3Yt...,SPRLD37U05,2023-06-27 19:49,NaN
2,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",3.0,40689378387,40689378387,0.0,0.0,0.0,NaN,50000000,...,40732301726/221844/20950092290/17890//3500//|4...,0,,"[{'nvMid': '40732301726', 'mallSeq': '221844',...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=%2Fds...,https://cr.shopping.naver.com/adcr.nhn?x=oXvSU...,SPRLD37U05,2023-06-27 19:49,NaN
0,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",1.0,39188787823,39188787823,1.0,0.0,0.0,4.9,50000000,...,40790343180/24/3024001978/21760//3000//|408039...,0,,"[{'nvMid': '40790343180', 'mallSeq': '24', 'ma...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=jCe7%...,https://cr.shopping.naver.com/adcr.nhn?x=RF4Tt...,SPYWD25C01,2023-06-27 19:49,NaN
1,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",2.0,40438127103,40438127103,0.0,0.0,0.0,NaN,50000000,...,40438308805/114/D399589991/30090//3000//|40345...,0,,"[{'nvMid': '40438308805', 'mallSeq': '114', 'm...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=P78de...,https://cr.shopping.naver.com/adcr.nhn?x=9%2FN...,SPYWD25C01,2023-06-27 19:49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",2.0,40670192704,40670192704,1.0,0.0,0.0,NaN,50000000,...,40525951507/221844/19081582190/19820//3000//|3...,0,,"[{'nvMid': '40525951507', 'mallSeq': '221844',...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=r9QNh...,https://cr.shopping.naver.com/adcr.nhn?x=AZTuG...,SPRWD23U02,2023-06-27 20:09,NaN
44,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",45.0,39813084005,39813084005,0.0,0.0,0.0,NaN,50000007,...,86026114321/7363534/8481613998/17000/101775847...,0,,"[{'nvMid': '86026114321', 'mallSeq': '7363534'...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=3o16%...,https://cr.shopping.naver.com/adcr.nhn?x=7A8im...,SPRWD23U02,2023-06-27 20:09,NaN
0,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",1.0,40498059346,40498059346,1.0,0.0,0.0,NaN,50000000,...,40323322697/24/2980269361/35320//3000//|402325...,0,,"[{'nvMid': '40323322697', 'mallSeq': '24', 'ma...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=4DeyF...,https://cr.shopping.naver.com/adcr.nhn?x=z2Z7r...,SPYWD26M06,2023-06-27 20:09,NaN
5,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",6.0,40806068123,40806068123,0.0,0.0,0.0,NaN,50000000,...,40629694692/221844/20618239782/41110//3000//|8...,0,,"[{'nvMid': '40629694692', 'mallSeq': '221844',...",NaN,https://cr.shopping.naver.com/adcr.nhn?x=fX1sW...,https://cr.shopping.naver.com/adcr.nhn?x=QXJO8...,SPYWD26M06,2023-06-27 20:09,NaN


In [ ]:
df_total = df_total.loc[~(df_total["lowMallList"].isnull() == False)]
# df_total

,collection,purchaseConditionInfos,rank,id,parentId,hasLowestCardPrice,hasPromotionCont,hasAddInFee,scoreInfo,category1Id,...,lowPriceByMallNo,shopNNo,dummy,lowMallList,mallInfoCache,crUrl,crUrlMore,styleCode,timestamp,channelInfoCache
0,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",1.0,40245729397,40245729397,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '393273', 'prodCnt': '344588', 'name':...",https://cr.shopping.naver.com/adcr.nhn?x=looAy...,https://cr.shopping.naver.com/adcr.nhn?x=uwlsk...,SPRPD37C02,2023-06-27 19:49,NaN
3,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",4.0,40566549639,40566549639,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '114', 'prodCnt': '182034701', 'name':...",https://cr.shopping.naver.com/adcr.nhn?x=EmrE7...,https://cr.shopping.naver.com/adcr.nhn?x=KlU1i...,SPRLD37U05,2023-06-27 19:49,NaN
4,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",5.0,40563125001,40563125001,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '3', 'prodCnt': '56555386', 'name': '인...",https://cr.shopping.naver.com/adcr.nhn?x=Q40jp...,https://cr.shopping.naver.com/adcr.nhn?x=71bXr...,SPRLD37U05,2023-06-27 19:49,NaN
5,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",6.0,40457021662,40127022611,1.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '17703', 'prodCnt': '78382887', 'name'...",https://cr.shopping.naver.com/adcr.nhn?x=qi3Ve...,https://cr.shopping.naver.com/adcr.nhn?x=lXwvB...,SPRLD37U05,2023-06-27 19:49,NaN
6,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",7.0,40366408044,40127022611,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '114', 'prodCnt': '182034701', 'name':...",https://cr.shopping.naver.com/adcr.nhn?x=LZmON...,https://cr.shopping.naver.com/adcr.nhn?x=amueL...,SPRLD37U05,2023-06-27 19:49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",112.0,40223003961,40498059346,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '221844', 'prodCnt': '52630386', 'name...",https://cr.shopping.naver.com/adcr.nhn?x=g9YAd...,https://cr.shopping.naver.com/adcr.nhn?x=cE71f...,SPYWD26M06,2023-06-27 20:09,NaN
112,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",113.0,40807540578,40498059346,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '24', 'prodCnt': '162903507', 'name': ...",https://cr.shopping.naver.com/adcr.nhn?x=ZsAzl...,https://cr.shopping.naver.com/adcr.nhn?x=ZbdFy...,SPYWD26M06,2023-06-27 20:09,NaN
113,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",114.0,40807544775,40498059346,0.0,0.0,0.0,NaN,50000000,...,,0,,None,"{'seq': '197023', 'prodCnt': '42056781', 'name...",https://cr.shopping.naver.com/adcr.nhn?x=%2FHM...,https://cr.shopping.naver.com/adcr.nhn?x=5DzZh...,SPYWD26M06,2023-06-27 20:09,NaN
114,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",115.0,86311591027,40498059346,0.0,0.0,0.0,NaN,50000000,...,,5763007,,None,"{'seq': '5763007', 'prodCnt': '5352', 'name': ...",https://cr.shopping.naver.com/adcr.nhn?x=zeq6a...,https://cr.shopping.naver.com/adcr.nhn?x=1rZZg...,SPYWD26M06,2023-06-27 20:09,{'talkAccountId': ''}


In [ ]:
df_total.to_csv(f"{path}total_{date}.csv", index=False)

# 2. 가격비교(df_model)

In [ ]:
# 가격 비교 내 상품 불러오는 함수
# pagingIndex=20, sort=LOW_PRICE

def productSetmodel(id):
    df = []
    pages = 1

    while True:
        url = f"https://search.shopping.naver.com/api/catalog/{id}/products?arrivalGuarantee=false&cardPrice=false&deliveryToday=false&fastDelivery=false&isNPayPlus=false&lowestPrice&nvMid={id}&onlyBeautyWindow=false&page={pages}&pageSize=20&pr=PC&sort=LOW_PRICE&withFee=false&catalogProviderTypeCode&catalogType=DEFAULT&exposeAreaName=SELLER_BY_PRICE&inflow=slc&isManual=false&query"
        response = requests.get(url,headers=headers)
        temp = response.json()['result']

        if temp['pagingCount'] == 0:
            break;

        df = df + temp["products"]
        pages = pages + 1

    return pd.DataFrame(df)

In [ ]:
df_model = pd.DataFrame()

for index, row in df_lowMallList.iterrows():
    id = row.id
    styleCode = row.styleCode

    df_temp = productSetmodel(id)
    # print(id)
    time.sleep(3)

    df_temp["styleCode"] = styleCode
    df_temp["timestamp"] = datetime.now(timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M")

    df_model = pd.concat([df_model, df_temp])

In [ ]:
# df_model

,nvMid,foldingCount,matchNvMid,categoryId,mallId,mallPid,mallSequence,mallName,channelSequence,channelName,...,mallGrade,mallBusinessNumber,mallRegisteredNumber,mallBusinessBaseAddress,mallUrl,updateDate,savingStorePoint,styleCode,timestamp,savingStoreCoupon
0,40417485820,2.0,40127022611,50000830,ticketmonster,20598021410,221844,티몬,None,None,...,PLATINUM,2118841856,제 2010 서울 강남 02066호,"서울특별시 강남구 압구정로 118 (신사동, 아리지 빌딩)",http://www.tmon.co.kr,2023.06.27.,NaN,SPRLD37U05,2023-06-27 20:10,NaN
1,40457021662,1.0,40127022611,50000830,11st,5878053145,17703,11번가,None,None,...,PLATINUM,8158101244,2018-서울중구-1445,"서울특별시 중구 한강대로 416 (남대문로5가, 서울스퀘어)",http://www.11st.co.kr,2023.06.27.,NaN,SPRLD37U05,2023-06-27 20:10,NaN
2,40741414253,1.0,40127022611,50000830,goodsdaq,3019094959,24,G마켓,None,None,...,PLATINUM,2208183676,강남 10630호,"서울특별시 강남구 테헤란로 152 (역삼동, 강남파이낸스센터)",http://www.gmarket.co.kr,2023.06.27.,NaN,SPRLD37U05,2023-06-27 20:10,NaN
3,40732336665,2.0,40127022611,50000830,ticketmonster,20950091974,221844,티몬,None,None,...,PLATINUM,2118841856,제 2010 서울 강남 02066호,"서울특별시 강남구 압구정로 118 (신사동, 아리지 빌딩)",http://www.tmon.co.kr,2023.06.27.,NaN,SPRLD37U05,2023-06-27 20:10,NaN
4,40232595157,2.0,40127022611,50000830,auction,D378627907,114,옥션,None,None,...,PLATINUM,2208183676,강남 10630호,"서울특별시 강남구 테헤란로 152 (역삼동, 강남파이낸스센터)",http://www.auction.co.kr,2023.06.27.,NaN,SPRLD37U05,2023-06-27 20:10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,40031951264,1.0,40498059346,50000830,wemakeprice,2527284860_2527284860,197023,위메프,None,None,...,PLATINUM,5668701096,제 2010 서울 강남 02066호,서울특별시 강남구 영동대로 502,http://www.wemakeprice.com,2023.06.27.,NaN,SPYWD26M06,2023-06-27 20:42,NaN
0,40629694692,2.0,40806068123,50000833,ticketmonster,20618239782,221844,티몬,None,None,...,PLATINUM,2118841856,제 2010 서울 강남 02066호,"서울특별시 강남구 압구정로 118 (신사동, 아리지 빌딩)",http://www.tmon.co.kr,2023.06.27.,NaN,SPYWD26M06,2023-06-27 20:42,NaN
1,86208380293,1.0,40806068123,50000830,ncp_1oj10d_01,8663879970,7273455,큐레이터A,101760185,큐레이터A,...,SEED,8773001269,2022-인천연수구-2029,인천광역시 연수구 송도미래로 30 (송도 BRC 스마트밸리 지식산업센터) B동 90...,https://smartstore.naver.com/curatora,2023.06.27.,NaN,SPYWD26M06,2023-06-27 20:42,NaN
0,86142342490,1.0,40412624349,50002630,ncp_1ojclr_01,8597842167,7363534,제일넘버원,101775847,제일넘버원,...,CHILDREN,7314300834,2022-부산동래-0946,부산광역시 동래구 중앙대로1335번길 17-1 (우 : 47734),https://smartstore.naver.com/jeilone,2023.06.27.,NaN,SPYWD26M06,2023-06-27 20:42,NaN


In [ ]:
df_model.to_csv(f"{path}model_{date}.csv", index=False)